## Data prep

In [23]:
import os
import time
import zipfile
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer

###############################################################################
# Utility functions

def extract_zip(zip_file, extract_to):
    """Extracts a zip file to the specified directory."""
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

def getDataPoints(path):
    """Collects content of all .py files within the given directory."""
    dataPointsList = []
    if not os.path.exists(path):
        print(f"Directory does not exist: {path}")
        return dataPointsList

    for root, dirs, files in os.walk(path):
        for dataPointName in files:
            if dataPointName.endswith(".py"):
                file_path = os.path.join(root, dataPointName)
                with open(file_path, encoding="utf-8") as fileIn:
                    dp = fileIn.read().strip()
                    if dp:  # Ensure the document is not empty
                        dataPointsList.append(dp)
                    else:
                        print(f"Empty or invalid file skipped: {file_path}")

    if len(dataPointsList) == 0:
        print(f"No valid documents found in directory: {path}")

    return dataPointsList

###############################################################################
# Data Extraction and Vectorization

# Parameters setup
flakyZip = "compressedDataset/flaky_files.zip"
nonFlakyZip = "compressedDataset/all_nonflaky_files.zip"

outDir = "smote-results"
os.makedirs(outDir, exist_ok=True)
extractDir = "smote-extracted"
os.makedirs(extractDir, exist_ok=True)

# Extract the zip files
flakyDir = os.path.join(extractDir, 'flaky')
nonFlakyDir = os.path.join(extractDir, 'nonFlaky')
os.makedirs(flakyDir, exist_ok=True)
os.makedirs(nonFlakyDir, exist_ok=True)

extract_zip(flakyZip, flakyDir)
extract_zip(nonFlakyZip, nonFlakyDir)

dataPointsFlaky = getDataPoints(flakyDir)
dataPointsNonFlaky = getDataPoints(nonFlakyDir)
dataPoints = dataPointsFlaky + dataPointsNonFlaky

print(f"Number of flaky documents: {len(dataPointsFlaky)}")
print(f"Number of non-flaky documents: {len(dataPointsNonFlaky)}")
print(f"Total number of documents: {len(dataPoints)}")

if len(dataPoints) == 0:
    raise ValueError("No documents available for vectorization. Please check the input directories.")

# Create labels: 1 for flaky, 0 for non-flaky
dataLabelsList = np.array([1]*len(dataPointsFlaky) + [0]*len(dataPointsNonFlaky))

Number of flaky documents: 45
Number of non-flaky documents: 254
Total number of documents: 299


## KNN

In [24]:
import os
import time
import numpy as np
import pandas as pd

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (
    precision_score, recall_score, accuracy_score, f1_score
)
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

def runKNNWithSMOTE(dataPoints, dataLabelsList, outDir, n_splits):
    # Start timing for preparation (if needed)
    # v0 = time.perf_counter()

    # Define scoring metrics
    scoring = {
        'precision': 'precision',
        'recall': 'recall',
        'accuracy': 'accuracy',
        'f1': 'f1'
    }

    # Define a pipeline with CountVectorizer, SMOTE, and KNN
    pipeline = ImbPipeline([
        ('vectorizer', CountVectorizer(stop_words=None)),  # Include vectorizer in pipeline
        ('smote', SMOTE(random_state=42)),                 # SMOTE for oversampling
        ('knn', KNeighborsClassifier())                    # KNN classifier
    ])

    # Define parameter grid for GridSearchCV
    param_grid = {
        'knn__n_neighbors': [3, 5, 7, 9, 11, 15, 20],      # Number of neighbors for KNN
        'knn__weights': ['uniform', 'distance'],           # Uniform or distance-based weighting
        'knn__metric': ['euclidean', 'cosine']             # Distance metrics
    }

    # Setup cross-validation strategy
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Setup GridSearchCV with the pipeline and parameter grid
    grid_search = GridSearchCV(
        pipeline, param_grid, cv=skf, scoring=scoring,
        refit='f1', verbose=1, return_train_score=True
    )

    # Fit the GridSearchCV to the data
    grid_search.fit(dataPoints, dataLabelsList)

    # Retrieve the best parameters and score from cross-validation
    best_params = grid_search.best_params_
    best_f1_cv = grid_search.best_score_

    print(f"Best Parameters with SMOTE: {best_params}")
    print(f"Best F1 Score from cross-validation: {best_f1_cv}")

    # Extract the cross-validation results and print final metrics
    results = grid_search.cv_results_

    # Prepare per-fold metrics for CSV along with the parameter combinations
    fold_metrics = []
    for idx in range(len(results['params'])):
        fold_metrics.append({
            'n_neighbors': results['params'][idx].get('knn__n_neighbors'),
            'weights': results['params'][idx].get('knn__weights'),
            'metric': results['params'][idx].get('knn__metric'),
            'accuracy': results['mean_test_accuracy'][idx],
            'precision': results['mean_test_precision'][idx],
            'recall': results['mean_test_recall'][idx],
            'f1': results['mean_test_f1'][idx],
            # Remove or adjust 'preparationTime' as 'vecTime' is undefined here
            # 'preparationTime': vecTime / len(dataLabelsList)
        })

    # Save fold-wise metrics to CSV
    df_folds = pd.DataFrame(fold_metrics)
    outFile = os.path.join(outDir, f"knn-smote-fold-results-{n_splits}-folds.csv")
    df_folds.to_csv(outFile, index=False)

    print(f"Per-fold metrics saved to: {outFile}")

    # Extract final metrics based on the cross-validation results
    final_f1 = results['mean_test_f1'][grid_search.best_index_]
    final_precision = results['mean_test_precision'][grid_search.best_index_]
    final_recall = results['mean_test_recall'][grid_search.best_index_]
    final_accuracy = results['mean_test_accuracy'][grid_search.best_index_]

    # Print final metrics (cross-validation averages)
    print("\nFinal Cross-Validation Metrics:")
    print(f"Final Precision: {final_precision}")
    print(f"Final Recall: {final_recall}")
    print(f"Final Accuracy: {final_accuracy}")
    print(f"Final F1 Score: {final_f1}")

    # Save the results to a CSV file
    outFile = os.path.join(outDir, f"knn-results-{n_splits}-folds.csv")
    with open(outFile, "w") as f:
        f.write("Accuracy,Precision,Recall,F1\n")
        f.write(f"{final_accuracy},{final_precision},{final_recall},{final_f1}\n")

    print(f"KNN analysis completed for {n_splits}-folds with SMOTE. Results saved to: {outFile}")

    return best_params, final_f1

###############################################################################
# Main Execution for Both 3-Fold and 5-Fold

# Ensure 'outDir' is defined
outDir = "smote-results"
os.makedirs(outDir, exist_ok=True)

# Run KNN with SMOTE using 3-fold cross-validation
print("Starting KNN analysis with SMOTE for 3-fold cross-validation...")
best_params_3folds, best_f1_3folds = runKNNWithSMOTE(dataPoints, dataLabelsList, outDir, n_splits=3)

# Run KNN with SMOTE using 5-fold cross-validation
print("\nStarting KNN analysis with SMOTE for 5-fold cross-validation...")
best_params_5folds, best_f1_5folds = runKNNWithSMOTE(dataPoints, dataLabelsList, outDir, n_splits=5)

# Display results
print("\nBest results for KNN with SMOTE 3-fold cross-validation:")
print(f"Best Parameters: {best_params_3folds}")
print(f"Best F1 Score: {best_f1_3folds}")

print("\nBest results for KNN with SMOTE 5-fold cross-validation:")
print(f"Best Parameters: {best_params_5folds}")
print(f"Best F1 Score: {best_f1_5folds}")


Starting KNN analysis with SMOTE for 3-fold cross-validation...
Fitting 3 folds for each of 28 candidates, totalling 84 fits
Best Parameters with SMOTE: {'knn__metric': 'cosine', 'knn__n_neighbors': 3, 'knn__weights': 'distance'}
Best F1 Score from cross-validation: 0.5264550264550265
Per-fold metrics saved to: smote-results\knn-smote-fold-results-3-folds.csv

Final Cross-Validation Metrics:
Final Precision: 0.4506172839506173
Final Recall: 0.6444444444444444
Final Accuracy: 0.822962962962963
Final F1 Score: 0.5264550264550265
KNN analysis completed for 3-folds with SMOTE. Results saved to: smote-results\knn-results-3-folds.csv

Starting KNN analysis with SMOTE for 5-fold cross-validation...
Fitting 5 folds for each of 28 candidates, totalling 140 fits
Best Parameters with SMOTE: {'knn__metric': 'cosine', 'knn__n_neighbors': 3, 'knn__weights': 'distance'}
Best F1 Score from cross-validation: 0.5605882352941176
Per-fold metrics saved to: smote-results\knn-smote-fold-results-5-folds.csv


## SVM

In [25]:
import numpy as np
import pandas as pd

from sklearn.svm import SVC
from sklearn.metrics import (
    precision_score, recall_score, accuracy_score, f1_score
)
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

def runSVMWithSMOTE(dataPoints, dataLabelsList, outDir, n_splits):
    v0 = time.perf_counter()

    # Define scoring metrics
    scoring = {
        'precision': 'precision',
        'recall': 'recall',
        'accuracy': 'accuracy',
        'f1': 'f1'
    }

    # Define a pipeline with SMOTE and SVM 
    pipeline = ImbPipeline([
        ('vectorizer', CountVectorizer(stop_words=None)),  # Include vectorizer in pipeline
        ('smote', SMOTE(random_state=42)),    # SMOTE for oversampling
        ('svm', SVC(probability=True))        # SVM classifier
    ])

    # Define parameter grid for GridSearchCV
    param_grid = {
        'svm__C': [0.01, 0.1, 1.0, 10.0, 100.0],    # Regularization parameter
        'svm__kernel': ['linear', 'rbf', 'poly', 'sigmoid']   # Kernel types
    }

    # Setup cross-validation strategy
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Setup GridSearchCV with the pipeline and parameter grid
    grid_search = GridSearchCV(pipeline, param_grid, cv=skf, scoring=scoring, refit='f1', verbose=1, return_train_score=True)

    # Fit the GridSearchCV to the data
    grid_search.fit(dataPoints, dataLabelsList)

    # Retrieve the best parameters and score from cross-validation
    best_params = grid_search.best_params_
    best_f1_cv = grid_search.best_score_

    print(f"Best Parameters with SMOTE: {best_params}")
    print(f"Best F1 Score from cross-validation: {best_f1_cv}")

    # Extract the cross-validation results and print final metrics
    results = grid_search.cv_results_

    # Prepare per-fold metrics for CSV along with the parameter combinations
    fold_metrics = []
    for idx in range(len(results['params'])):
        fold_metrics.append({
            'C': results['params'][idx].get('svm__C'),
            'kernel': results['params'][idx].get('svm__kernel'),
            'accuracy': results['mean_test_accuracy'][idx],
            'precision': results['mean_test_precision'][idx],
            'recall': results['mean_test_recall'][idx],
            'f1': results['mean_test_f1'][idx],
        })

    # Save fold-wise metrics to CSV
    df_folds = pd.DataFrame(fold_metrics)
    outFile = os.path.join(outDir, f"svm-smote-fold-results-{n_splits}-folds.csv")
    df_folds.to_csv(outFile, index=False)

    print(f"Per-fold metrics saved to: {outFile}")

    # Extract final metrics based on the cross-validation results
    final_f1 = results['mean_test_f1'][grid_search.best_index_]
    final_precision = results['mean_test_precision'][grid_search.best_index_]
    final_recall = results['mean_test_recall'][grid_search.best_index_]
    final_accuracy = results['mean_test_accuracy'][grid_search.best_index_]

    # Print final metrics (cross-validation averages)
    print("\nFinal Cross-Validation Metrics:")
    print(f"Final Precision: {final_precision}")
    print(f"Final Recall: {final_recall}")
    print(f"Final Accuracy: {final_accuracy}")
    print(f"Final F1 Score: {final_f1}")

    # Save the results to a CSV file
    outFile = os.path.join(outDir, f"svm-results-{n_splits}-folds.csv")
    with open(outFile, "w") as f:
        f.write("Accuracy,Precision,Recall,F1\n")
        f.write(f"{final_accuracy},{final_precision},{final_recall},{final_f1}\n")

    print(f"SVM analysis completed for {n_splits}-folds with SMOTE. Results saved to: {outFile}")

    return best_params, final_f1

###############################################################################
# Main Execution for Both 3-Fold and 5-Fold

# Run SVM with SMOTE using 3-fold cross-validation
print("Starting SVM analysis with SMOTE for 3-fold cross-validation...")
best_params_3folds, best_f1_3folds = runSVMWithSMOTE(dataPoints, dataLabelsList, outDir, n_splits=3)

# Run SVM with SMOTE using 5-fold cross-validation
print("\nStarting SVM analysis with SMOTE for 5-fold cross-validation...")
best_params_5folds, best_f1_5folds = runSVMWithSMOTE(dataPoints, dataLabelsList, outDir, n_splits=5)

# Display results
print("\nBest results for SVM with SMOTE 3-fold cross-validation:")
print(f"Best Parameters: {best_params_3folds}")
print(f"Best F1 Score: {best_f1_3folds}")

print("\nBest results for SVM with SMOTE 5-fold cross-validation:")
print(f"Best Parameters: {best_params_5folds}")
print(f"Best F1 Score: {best_f1_5folds}")


Starting SVM analysis with SMOTE for 3-fold cross-validation...
Fitting 3 folds for each of 20 candidates, totalling 60 fits


c:\Users\haha9\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Best Parameters with SMOTE: {'svm__C': 0.01, 'svm__kernel': 'linear'}
Best F1 Score from cross-validation: 0.6498745819397994
Per-fold metrics saved to: smote-results\svm-smote-fold-results-3-folds.csv

Final Cross-Validation Metrics:
Final Precision: 0.7473262032085563
Final Recall: 0.6
Final Accuracy: 0.9096296296296296
Final F1 Score: 0.6498745819397994
SVM analysis completed for 3-folds with SMOTE. Results saved to: smote-results\svm-results-3-folds.csv

Starting SVM analysis with SMOTE for 5-fold cross-validation...
Fitting 5 folds for each of 20 candidates, totalling 100 fits


c:\Users\haha9\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\haha9\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\haha9\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capita

Best Parameters with SMOTE: {'svm__C': 0.1, 'svm__kernel': 'linear'}
Best F1 Score from cross-validation: 0.6234188034188035
Per-fold metrics saved to: smote-results\svm-smote-fold-results-5-folds.csv

Final Cross-Validation Metrics:
Final Precision: 0.7247474747474747
Final Recall: 0.5777777777777777
Final Accuracy: 0.9029378531073448
Final F1 Score: 0.6234188034188035
SVM analysis completed for 5-folds with SMOTE. Results saved to: smote-results\svm-results-5-folds.csv

Best results for SVM with SMOTE 3-fold cross-validation:
Best Parameters: {'svm__C': 0.01, 'svm__kernel': 'linear'}
Best F1 Score: 0.6498745819397994

Best results for SVM with SMOTE 5-fold cross-validation:
Best Parameters: {'svm__C': 0.1, 'svm__kernel': 'linear'}
Best F1 Score: 0.6234188034188035


## Naive Bayes

In [26]:
import numpy as np
import pandas as pd

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import (
    precision_score, recall_score, accuracy_score, f1_score
)
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

def runNBWithSMOTE(dataPoints, dataLabelsList, outDir, n_splits):
    v0 = time.perf_counter()

    # Define scoring metrics
    scoring = {
        'precision': 'precision',
        'recall': 'recall',
        'accuracy': 'accuracy',
        'f1': 'f1'
    }

    pipeline = ImbPipeline([
        ('vectorizer', CountVectorizer(stop_words=None)),  # Include vectorizer in pipeline
        ('smote', SMOTE(random_state=42)),    # SMOTE for oversampling
        ('nb', MultinomialNB())               # Naive Bayes classifier
    ])

    # Define parameter grid for GridSearchCV
    param_grid = {
        'nb__alpha': [0.001, 0.01, 0.1, 1.0, 10.0]   # Smoothing parameter for Naive Bayes
    }

    # Setup cross-validation strategy
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Setup GridSearchCV with the pipeline and parameter grid
    grid_search = GridSearchCV(pipeline, param_grid, cv=skf, scoring=scoring, refit='f1', verbose=1, return_train_score=True)

    # Fit the GridSearchCV to the data
    grid_search.fit(dataPoints, dataLabelsList)

    # Retrieve the best parameters and score from cross-validation
    best_params = grid_search.best_params_
    best_f1_cv = grid_search.best_score_

    print(f"Best Parameters with SMOTE: {best_params}")
    print(f"Best F1 Score from cross-validation: {best_f1_cv}")

    # Extract the cross-validation results and print final metrics
    results = grid_search.cv_results_

    # Prepare per-fold metrics for CSV along with the parameter combinations
    fold_metrics = []
    for idx in range(len(results['params'])):
        fold_metrics.append({
            'alpha': results['params'][idx].get('nb__alpha'),
            'accuracy': results['mean_test_accuracy'][idx],
            'precision': results['mean_test_precision'][idx],
            'recall': results['mean_test_recall'][idx],
            'f1': results['mean_test_f1'][idx],
        })

    # Save fold-wise metrics to CSV
    df_folds = pd.DataFrame(fold_metrics)
    outFile = os.path.join(outDir, f"nb-smote-fold-results-{n_splits}-folds.csv")
    df_folds.to_csv(outFile, index=False)

    print(f"Per-fold metrics saved to: {outFile}")

    # Extract final metrics based on the cross-validation results
    final_f1 = results['mean_test_f1'][grid_search.best_index_]
    final_precision = results['mean_test_precision'][grid_search.best_index_]
    final_recall = results['mean_test_recall'][grid_search.best_index_]
    final_accuracy = results['mean_test_accuracy'][grid_search.best_index_]

    # Print final metrics (cross-validation averages)
    print("\nFinal Cross-Validation Metrics:")
    print(f"Final Precision: {final_precision}")
    print(f"Final Recall: {final_recall}")
    print(f"Final Accuracy: {final_accuracy}")
    print(f"Final F1 Score: {final_f1}")

    # Save the results to a CSV file
    outFile = os.path.join(outDir, f"nb-results-{n_splits}-folds.csv")
    with open(outFile, "w") as f:
        f.write("Accuracy,Precision,Recall,F1\n")
        f.write(f"{final_accuracy},{final_precision},{final_recall},{final_f1}\n")

    print(f"Naive Bayes analysis completed for {n_splits}-folds with SMOTE. Results saved to: {outFile}")

    return best_params, final_f1

###############################################################################
# Main Execution for Both 3-Fold and 5-Fold

# Run Naive Bayes with SMOTE using 3-fold cross-validation
print("Starting Naive Bayes analysis with SMOTE for 3-fold cross-validation...")
best_params_3folds, best_f1_3folds = runNBWithSMOTE(dataPoints, dataLabelsList, outDir, n_splits=3)

# Run Naive Bayes with SMOTE using 5-fold cross-validation
print("\nStarting Naive Bayes analysis with SMOTE for 5-fold cross-validation...")
best_params_5folds, best_f1_5folds = runNBWithSMOTE(dataPoints, dataLabelsList, outDir, n_splits=5)

# Display results
print("\nBest results for Naive Bayes with SMOTE 3-fold cross-validation:")
print(f"Best Parameters: {best_params_3folds}")
print(f"Best F1 Score: {best_f1_3folds}")

print("\nBest results for Naive Bayes with SMOTE 5-fold cross-validation:")
print(f"Best Parameters: {best_params_5folds}")
print(f"Best F1 Score: {best_f1_5folds}")


Starting Naive Bayes analysis with SMOTE for 3-fold cross-validation...
Fitting 3 folds for each of 5 candidates, totalling 15 fits
Best Parameters with SMOTE: {'nb__alpha': 1.0}
Best F1 Score from cross-validation: 0.6524712002972873
Per-fold metrics saved to: smote-results\nb-smote-fold-results-3-folds.csv

Final Cross-Validation Metrics:
Final Precision: 0.7785353535353535
Final Recall: 0.5777777777777778
Final Accuracy: 0.9097306397306397
Final F1 Score: 0.6524712002972873
Naive Bayes analysis completed for 3-folds with SMOTE. Results saved to: smote-results\nb-results-3-folds.csv

Starting Naive Bayes analysis with SMOTE for 5-fold cross-validation...
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best Parameters with SMOTE: {'nb__alpha': 0.001}
Best F1 Score from cross-validation: 0.6589043309631546
Per-fold metrics saved to: smote-results\nb-smote-fold-results-5-folds.csv

Final Cross-Validation Metrics:
Final Precision: 0.795
Final Recall: 0.5777777777777777
Final 

## XGB

In [27]:
import numpy as np
import pandas as pd

from xgboost import XGBClassifier
from sklearn.metrics import (
    precision_score, recall_score, accuracy_score, f1_score
)
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

def runXGBWithSMOTE(dataPoints, dataLabelsList, outDir, n_splits):
    v0 = time.perf_counter()

    # Define scoring metrics
    scoring = {
        'precision': 'precision',
        'recall': 'recall',
        'accuracy': 'accuracy',
        'f1': 'f1'
    }

    # Define a pipeline with SMOTE and XGBoost (CountVectorizer is removed as data is already vectorized)
    pipeline = ImbPipeline([
        ('vectorizer', CountVectorizer(stop_words=None)),  # Include vectorizer in pipeline
        ('smote', SMOTE(random_state=42)),    # SMOTE for oversampling
        ('xgb', XGBClassifier(eval_metric='logloss', random_state=42))  # XGBoost classifier
    ])

    # Define parameter grid for GridSearchCV
    param_grid = {
        'xgb__n_estimators': [100, 150, 200, 300],   # Number of boosting rounds
        'xgb__max_depth': [3, 5, 7, 10],             # Maximum depth of a tree
        'xgb__learning_rate': [0.01, 0.1, 0.3, 0.5], # Learning rate
    }

    # Setup cross-validation strategy
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Setup GridSearchCV with the pipeline and parameter grid
    grid_search = GridSearchCV(pipeline, param_grid, cv=skf, scoring=scoring, refit='f1', verbose=1, return_train_score=True)

    # Fit the GridSearchCV to the data
    grid_search.fit(dataPoints, dataLabelsList)

    # Retrieve the best parameters and score from cross-validation
    best_params = grid_search.best_params_
    best_f1_cv = grid_search.best_score_

    print(f"Best Parameters with SMOTE: {best_params}")
    print(f"Best F1 Score from cross-validation: {best_f1_cv}")

    # Extract the cross-validation results and print final metrics
    results = grid_search.cv_results_

    # Prepare per-fold metrics for CSV along with the parameter combination
    fold_metrics = []
    for idx in range(len(results['params'])):
        fold_metrics.append({
            'n_estimators': results['params'][idx].get('xgb__n_estimators'),
            'max_depth': results['params'][idx].get('xgb__max_depth'),
            'learning_rate': results['params'][idx].get('xgb__learning_rate'),
            'accuracy': results['mean_test_accuracy'][idx],
            'precision': results['mean_test_precision'][idx],
            'recall': results['mean_test_recall'][idx],
            'f1': results['mean_test_f1'][idx],
        })

    # Save fold-wise metrics to CSV
    df_folds = pd.DataFrame(fold_metrics)
    outFile = os.path.join(outDir, f"xgb-smote-fold-results-{n_splits}-folds.csv")
    df_folds.to_csv(outFile, index=False)

    print(f"Per-fold metrics saved to: {outFile}")

    # Extract final metrics based on the cross-validation results
    final_f1 = results['mean_test_f1'][grid_search.best_index_]
    final_precision = results['mean_test_precision'][grid_search.best_index_]
    final_recall = results['mean_test_recall'][grid_search.best_index_]
    final_accuracy = results['mean_test_accuracy'][grid_search.best_index_]

    # Print final metrics (cross-validation averages)
    print("\nFinal Cross-Validation Metrics:")
    print(f"Final Precision: {final_precision}")
    print(f"Final Recall: {final_recall}")
    print(f"Final Accuracy: {final_accuracy}")
    print(f"Final F1 Score: {final_f1}")

    # Save the results to a CSV file
    outFile = os.path.join(outDir, f"xgb-results-{n_splits}-folds.csv")
    with open(outFile, "w") as f:
        f.write("Accuracy,Precision,Recall,F1\n")
        f.write(f"{final_accuracy},{final_precision},{final_recall},{final_f1}\n")

    print(f"XGBoost analysis completed for {n_splits}-folds with SMOTE. Results saved to: {outFile}")

    return best_params, final_f1

###############################################################################
# Main Execution for Both 3-Fold and 5-Fold

# Run XGBoost with SMOTE using 3-fold cross-validation
print("Starting XGBoost analysis with SMOTE for 3-fold cross-validation...")
best_params_3folds, best_f1_3folds = runXGBWithSMOTE(dataPoints, dataLabelsList, outDir, n_splits=3)

# Run XGBoost with SMOTE using 5-fold cross-validation
print("\nStarting XGBoost analysis with SMOTE for 5-fold cross-validation...")
best_params_5folds, best_f1_5folds = runXGBWithSMOTE(dataPoints, dataLabelsList, outDir, n_splits=5)

# Display results
print("\nBest results for XGBoost with SMOTE 3-fold cross-validation:")
print(f"Best Parameters: {best_params_3folds}")
print(f"Best F1 Score: {best_f1_3folds}")

print("\nBest results for XGBoost with SMOTE 5-fold cross-validation:")
print(f"Best Parameters: {best_params_5folds}")
print(f"Best F1 Score: {best_f1_5folds}")


Starting XGBoost analysis with SMOTE for 3-fold cross-validation...
Fitting 3 folds for each of 64 candidates, totalling 192 fits
Best Parameters with SMOTE: {'xgb__learning_rate': 0.1, 'xgb__max_depth': 3, 'xgb__n_estimators': 300}
Best F1 Score from cross-validation: 0.884802043422733
Per-fold metrics saved to: smote-results\xgb-smote-fold-results-3-folds.csv

Final Cross-Validation Metrics:
Final Precision: 1.0
Final Recall: 0.8000000000000002
Final Accuracy: 0.9698989898989899
Final F1 Score: 0.884802043422733
XGBoost analysis completed for 3-folds with SMOTE. Results saved to: smote-results\xgb-results-3-folds.csv

Starting XGBoost analysis with SMOTE for 5-fold cross-validation...
Fitting 5 folds for each of 64 candidates, totalling 320 fits
Best Parameters with SMOTE: {'xgb__learning_rate': 0.1, 'xgb__max_depth': 7, 'xgb__n_estimators': 300}
Best F1 Score from cross-validation: 0.86609243697479
Per-fold metrics saved to: smote-results\xgb-smote-fold-results-5-folds.csv

Final Cr

## Random Forest

In [28]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    precision_score, recall_score, accuracy_score, f1_score
)
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

def runRFWithSMOTE(dataPoints, dataLabelsList, outDir, n_splits):
    v0 = time.perf_counter()

    # Define scoring metrics
    scoring = {
        'precision': 'precision',
        'recall': 'recall',
        'accuracy': 'accuracy',
        'f1': 'f1'
    }

    # Define a pipeline with SMOTE and Random Forest
    pipeline = ImbPipeline([
        ('vectorizer', CountVectorizer(stop_words=None)),  # Include vectorizer in pipeline
        ('smote', SMOTE(random_state=42)),                 # SMOTE for oversampling
        ('rf', RandomForestClassifier(random_state=42))    # Random Forest classifier
    ])

    # Define parameter grid for GridSearchCV
    param_grid = {
        'rf__n_estimators': [10, 50, 100],               # Number of trees in the forest
        'rf__max_depth': [10, 30, 50],                   # Maximum depth of the tree
        'rf__min_samples_split': [2, 5],                 # Minimum number of samples required to split a node
        'rf__min_samples_leaf': [1, 2],                  # Minimum number of samples required at a leaf node
        'rf__criterion': ['gini', 'entropy']             # Function to measure the quality of a split
    }

    # Setup cross-validation strategy
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Setup GridSearchCV with the pipeline and parameter grid
    grid_search = GridSearchCV(pipeline, param_grid, cv=skf, scoring=scoring, refit='f1', verbose=1, return_train_score=True)

    # Fit the GridSearchCV to the data
    grid_search.fit(dataPoints, dataLabelsList)

    # Retrieve the best parameters and score from cross-validation
    best_params = grid_search.best_params_
    best_f1_cv = grid_search.best_score_

    print(f"Best Parameters with SMOTE: {best_params}")
    print(f"Best F1 Score from cross-validation: {best_f1_cv}")

    # Extract the cross-validation results and print final metrics
    results = grid_search.cv_results_

    # Prepare per-fold metrics for CSV along with the parameter combinations
    fold_metrics = []
    for idx in range(len(results['params'])):
        fold_metrics.append({
            'n_estimators': results['params'][idx].get('rf__n_estimators'),
            'max_depth': results['params'][idx].get('rf__max_depth'),
            'min_samples_split': results['params'][idx].get('rf__min_samples_split'),
            'min_samples_leaf': results['params'][idx].get('rf__min_samples_leaf'),
            'criterion': results['params'][idx].get('rf__criterion'),
            'accuracy': results['mean_test_accuracy'][idx],
            'precision': results['mean_test_precision'][idx],
            'recall': results['mean_test_recall'][idx],
            'f1': results['mean_test_f1'][idx],
        })

    # Save fold-wise metrics to CSV
    df_folds = pd.DataFrame(fold_metrics)
    outFile = os.path.join(outDir, f"rf-smote-fold-results-{n_splits}-folds.csv")
    df_folds.to_csv(outFile, index=False)

    print(f"Per-fold metrics saved to: {outFile}")

    # Extract final metrics based on the cross-validation results
    final_f1 = results['mean_test_f1'][grid_search.best_index_]
    final_precision = results['mean_test_precision'][grid_search.best_index_]
    final_recall = results['mean_test_recall'][grid_search.best_index_]
    final_accuracy = results['mean_test_accuracy'][grid_search.best_index_]

    # Print final metrics (cross-validation averages)
    print("\nFinal Cross-Validation Metrics:")
    print(f"Final Precision: {final_precision}")
    print(f"Final Recall: {final_recall}")
    print(f"Final Accuracy: {final_accuracy}")
    print(f"Final F1 Score: {final_f1}")

    # Save the results to a CSV file
    outFile = os.path.join(outDir, f"rf-results-{n_splits}-folds.csv")
    with open(outFile, "w") as f:
        f.write("Accuracy,Precision,Recall,F1\n")
        f.write(f"{final_accuracy},{final_precision},{final_recall},{final_f1}\n")

    print(f"Random Forest analysis completed for {n_splits}-folds with SMOTE. Results saved to: {outFile}")

    return best_params, final_f1

###############################################################################
# Main Execution for Both 3-Fold and 5-Fold

# Run Random Forest with SMOTE using 3-fold cross-validation
print("Starting Random Forest analysis with SMOTE for 3-fold cross-validation...")
best_params_3folds, best_f1_3folds = runRFWithSMOTE(dataPoints, dataLabelsList, outDir, n_splits=3)

# Run Random Forest with SMOTE using 5-fold cross-validation
print("\nStarting Random Forest analysis with SMOTE for 5-fold cross-validation...")
best_params_5folds, best_f1_5folds = runRFWithSMOTE(dataPoints, dataLabelsList, outDir, n_splits=5)

# Display results
print("\nBest results for Random Forest with SMOTE 3-fold cross-validation:")
print(f"Best Parameters: {best_params_3folds}")
print(f"Best F1 Score: {best_f1_3folds}")

print("\nBest results for Random Forest with SMOTE 5-fold cross-validation:")
print(f"Best Parameters: {best_params_5folds}")
print(f"Best F1 Score: {best_f1_5folds}")




Starting Random Forest analysis with SMOTE for 3-fold cross-validation...
Fitting 3 folds for each of 72 candidates, totalling 216 fits
Best Parameters with SMOTE: {'rf__criterion': 'entropy', 'rf__max_depth': 30, 'rf__min_samples_leaf': 2, 'rf__min_samples_split': 2, 'rf__n_estimators': 100}
Best F1 Score from cross-validation: 0.8099616858237547
Per-fold metrics saved to: smote-results\rf-smote-fold-results-3-folds.csv

Final Cross-Validation Metrics:
Final Precision: 0.8206349206349207
Final Recall: 0.7999999999999999
Final Accuracy: 0.943063973063973
Final F1 Score: 0.8099616858237547
Random Forest analysis completed for 3-folds with SMOTE. Results saved to: smote-results\rf-results-3-folds.csv

Starting Random Forest analysis with SMOTE for 5-fold cross-validation...
Fitting 5 folds for each of 72 candidates, totalling 360 fits
Best Parameters with SMOTE: {'rf__criterion': 'entropy', 'rf__max_depth': 30, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 5, 'rf__n_estimators': 10

## Decision Tree

In [29]:
import numpy as np
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (
    precision_score, recall_score, accuracy_score, f1_score
)
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

def runDTWithSMOTE(dataPoints, dataLabelsList, outDir, n_splits):
    v0 = time.perf_counter()

    # Define scoring metrics
    scoring = {
        'precision': 'precision',
        'recall': 'recall',
        'accuracy': 'accuracy',
        'f1': 'f1'
    }

    # Define a pipeline with SMOTE and Decision Tree
    pipeline = ImbPipeline([
        ('vectorizer', CountVectorizer(stop_words=None)),  # Include vectorizer in pipeline
        ('smote', SMOTE(random_state=42)),                 # SMOTE for oversampling
        ('dt', DecisionTreeClassifier(random_state=42))    # Decision Tree classifier
    ])

    # Define parameter grid for GridSearchCV
    param_grid = {
        'dt__max_depth': [10, 30, 50],                   # Maximum depth of the tree
        'dt__min_samples_split': [2, 5],                 # Minimum number of samples required to split a node
        'dt__min_samples_leaf': [1, 2],                  # Minimum number of samples required at a leaf node
        'dt__criterion': ['gini', 'entropy']             # Function to measure the quality of a split
    }

    # Setup cross-validation strategy
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Setup GridSearchCV with the pipeline and parameter grid
    grid_search = GridSearchCV(
        pipeline, param_grid, cv=skf, scoring=scoring,
        refit='f1', verbose=1, return_train_score=True
    )

    # Fit the GridSearchCV to the data
    grid_search.fit(dataPoints, dataLabelsList)

    # Retrieve the best parameters and score from cross-validation
    best_params = grid_search.best_params_
    best_f1_cv = grid_search.best_score_

    print(f"Best Parameters with SMOTE: {best_params}")
    print(f"Best F1 Score from cross-validation: {best_f1_cv}")

    # Extract the cross-validation results and print final metrics
    results = grid_search.cv_results_

    # Calculate preparation time per document
    vecTime = time.perf_counter() - v0

    # Prepare per-fold metrics for CSV along with the parameter combinations
    fold_metrics = []
    for idx in range(len(results['params'])):
        fold_metrics.append({
            'max_depth': results['params'][idx].get('dt__max_depth'),
            'min_samples_split': results['params'][idx].get('dt__min_samples_split'),
            'min_samples_leaf': results['params'][idx].get('dt__min_samples_leaf'),
            'criterion': results['params'][idx].get('dt__criterion'),
            'accuracy': results['mean_test_accuracy'][idx],
            'precision': results['mean_test_precision'][idx],
            'recall': results['mean_test_recall'][idx],
            'f1': results['mean_test_f1'][idx],
        })

    # Save fold-wise metrics to CSV
    df_folds = pd.DataFrame(fold_metrics)
    outFile = os.path.join(outDir, f"dt-smote-fold-results-{n_splits}-folds.csv")
    df_folds.to_csv(outFile, index=False)

    print(f"Per-fold metrics saved to: {outFile}")

    # Extract final metrics based on the cross-validation results
    final_f1 = grid_search.best_score_
    final_precision = results['mean_test_precision'][grid_search.best_index_]
    final_recall = results['mean_test_recall'][grid_search.best_index_]
    final_accuracy = results['mean_test_accuracy'][grid_search.best_index_]

    # Print final metrics (cross-validation averages)
    print("\nFinal Cross-Validation Metrics:")
    print(f"Final Precision: {final_precision}")
    print(f"Final Recall: {final_recall}")
    print(f"Final Accuracy: {final_accuracy}")
    print(f"Final F1 Score: {final_f1}")

    # Save the results to a CSV file
    outFile = os.path.join(outDir, f"dt-results-{n_splits}-folds.csv")
    with open(outFile, "w") as f:
        f.write("Accuracy,Precision,Recall,F1\n")
        f.write(f"{final_accuracy},{final_precision},{final_recall},{final_f1}\n")

    print(f"Decision Tree analysis completed for {n_splits}-folds with SMOTE. Results saved to: {outFile}")

    return best_params, final_f1

###############################################################################
# Main Execution for Both 3-Fold and 5-Fold

# Assuming 'Z', 'dataLabelsList', and 'outDir' are already defined

# Run Decision Tree with SMOTE using 3-fold cross-validation
print("Starting Decision Tree analysis with SMOTE for 3-fold cross-validation...")
best_params_3folds, best_f1_3folds = runDTWithSMOTE(dataPoints, dataLabelsList, outDir, n_splits=3)

# Run Decision Tree with SMOTE using 5-fold cross-validation
print("\nStarting Decision Tree analysis with SMOTE for 5-fold cross-validation...")
best_params_5folds, best_f1_5folds = runDTWithSMOTE(dataPoints, dataLabelsList, outDir, n_splits=5)

# Display results
print("\nBest results for Decision Tree with SMOTE 3-fold cross-validation:")
print(f"Best Parameters: {best_params_3folds}")
print(f"Best F1 Score: {best_f1_3folds}")

print("\nBest results for Decision Tree with SMOTE 5-fold cross-validation:")
print(f"Best Parameters: {best_params_5folds}")
print(f"Best F1 Score: {best_f1_5folds}")


Starting Decision Tree analysis with SMOTE for 3-fold cross-validation...
Fitting 3 folds for each of 24 candidates, totalling 72 fits
Best Parameters with SMOTE: {'dt__criterion': 'entropy', 'dt__max_depth': 10, 'dt__min_samples_leaf': 2, 'dt__min_samples_split': 2}
Best F1 Score from cross-validation: 0.8475533661740559
Per-fold metrics saved to: smote-results\dt-smote-fold-results-3-folds.csv

Final Cross-Validation Metrics:
Final Precision: 0.9410256410256409
Final Recall: 0.7777777777777777
Final Accuracy: 0.9598653198653198
Final F1 Score: 0.8475533661740559
Decision Tree analysis completed for 3-folds with SMOTE. Results saved to: smote-results\dt-results-3-folds.csv

Starting Decision Tree analysis with SMOTE for 5-fold cross-validation...
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Best Parameters with SMOTE: {'dt__criterion': 'gini', 'dt__max_depth': 10, 'dt__min_samples_leaf': 1, 'dt__min_samples_split': 2}
Best F1 Score from cross-validation: 0.83130374957

In [30]:
import os
import pandas as pd

# Function to extract the best results from the CSV files of each model
def extract_best_results(model_name, fold, csv_file):
    """
    Extracts the best result from the CSV file for a model.

    Parameters:
    - model_name: The name of the model (e.g., "XGBoost", "Random Forest", "Decision Tree")
    - fold: Number of folds (e.g., 5 or 3)
    - csv_file: The path to the CSV file containing the model's results

    Returns:
    A dictionary containing the best results for the model and fold.
    """
    if not os.path.exists(csv_file):
        print(f"CSV file for {model_name} ({fold}-fold) does not exist: {csv_file}")
        return None

    # Read the CSV file
    df = pd.read_csv(csv_file)

    # Get the row with the best F1 score
    best_row = df.loc[df['f1'].idxmax()]

    # Collect the best results into a dictionary
    best_results = {
        'Model': model_name,
        'Fold': f"{fold}-fold",
        'Best Accuracy': best_row['accuracy'],
        'Best Precision': best_row['precision'],
        'Best Recall': best_row['recall'],
        'Best F1 Score': best_row['f1'],
        'Best Parameters': best_row.to_dict()  # Including all parameters
    }

    return best_results

# Function to gather and print/save the best results from all SMOTE models
def gather_best_results(models_results_dir, output_file):
    """
    Gathers the best results from all SMOTE models and writes them to a CSV file.

    Parameters:
    - models_results_dir: Directory where the model result CSV files are stored for the SMOTE models.
    - output_file: Path to the output CSV file to store the best results.
    """
    # List of models and their corresponding result files for both 5-fold and 3-fold
    models = {
        'KNN': {'5-fold': 'knn-smote-fold-results-5-folds.csv', '3-fold': 'knn-smote-fold-results-3-folds.csv'},
        'SVM': {'5-fold': 'svm-smote-fold-results-5-folds.csv', '3-fold': 'svm-smote-fold-results-5-folds.csv'},
        'Naive Bayes': {'5-fold': 'nb-smote-fold-results-5-folds.csv', '3-fold': 'nb-smote-fold-results-3-folds.csv'},
        'XGBoost': {'5-fold': 'xgb-smote-fold-results-5-folds.csv', '3-fold': 'xgb-smote-fold-results-3-folds.csv'},
        'Random Forest': {'5-fold': 'rf-smote-fold-results-5-folds.csv', '3-fold': 'rf-smote-fold-results-3-folds.csv'},
        'Decision Tree': {'5-fold': 'dt-smote-fold-results-5-folds.csv', '3-fold': 'dt-smote-fold-results-3-folds.csv'}
    }

    # Initialize an empty list to store the best results from each model and fold
    best_results = []

    # Iterate over each model and its result files for both 5-fold and 3-fold
    for model_name, folds in models.items():
        for fold, csv_file in folds.items():
            full_csv_path = os.path.join(models_results_dir, csv_file)
            best_result = extract_best_results(model_name, fold, full_csv_path)
            if best_result:
                best_results.append(best_result)

    # Convert the list of best results into a DataFrame
    best_results_df = pd.DataFrame(best_results)

    # Save the best results to the output CSV file
    best_results_df.to_csv(output_file, index=False)
    print(f"Best results saved to: {output_file}")

    # Print the best results as a table
    print(f"\nBest Results from All SMOTE Models")
    print(best_results_df.to_string(index=False))

# Example usage
if __name__ == "__main__":
    # Directories where the model result CSV files are stored
    results_dir = 'smote-results'

    # Path to the output CSV file where best results will be stored
    output_file = "best_results_smote.csv"

    # Gather and save the best results
    gather_best_results(results_dir, output_file)


Best results saved to: best_results_smote.csv

Best Results from All SMOTE Models
        Model        Fold  Best Accuracy  Best Precision  Best Recall  Best F1 Score                                                                                                                                                                                                                        Best Parameters
          KNN 5-fold-fold       0.849661        0.508939     0.644444       0.560588                                                 {'n_neighbors': 3, 'weights': 'distance', 'metric': 'cosine', 'accuracy': 0.8496610169491525, 'precision': 0.5089393939393939, 'recall': 0.6444444444444444, 'f1': 0.5605882352941176}
          KNN 3-fold-fold       0.822963        0.450617     0.644444       0.526455                                                  {'n_neighbors': 3, 'weights': 'distance', 'metric': 'cosine', 'accuracy': 0.822962962962963, 'precision': 0.4506172839506173, 'recall': 0.64444444444444